In [16]:
from elasticsearch7 import Elasticsearch
from elasticsearch7.helpers import scan
from tqdm import tqdm
import math
import pandas as pd

In [3]:
INDEX_NAME = "crawler"
es = Elasticsearch(cloud_id= "0feeb24636464a578a9c7a1ce9739181:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQyMzcyNjZmYzcwMzg0ZTA2OTM1MTJkZGIxMDgzYTRmMyQ1N2RhZjIzZTNiMWM0MjAwYjBhMDQ0MGY1ZTEyZTc2Yw==",
                   http_auth=("elastic", "pETnMazDlmfyCT2rZ2NAWh2V"))

es.ping()

True

In [4]:
inlinks_dict = {}
outlinks_dict = {}
inlinks_len = {}
outlinks_len = {}   

scroll_size = 1000
# Initialize scroll
scroll = scan(es, index=INDEX_NAME, query={"query": {"match_all": {}}}, scroll='5m', size=scroll_size)

# Iterate through scroll
for result in tqdm(scroll):
    # Extract document ID (assuming URL is the ID)
    doc_id = result['_id']
    
    # Extract inlinks and outlinks from the current result
    inlinks = result['_source'].get('inlinks', [])
    outlinks = result['_source'].get('outlinks', [])
    
    # Store inlinks and outlinks in separate dictionaries
    inlinks_dict[doc_id] = inlinks
    outlinks_dict[doc_id] = outlinks
    inlinks_len[doc_id] = len(inlinks)
    outlinks_len[doc_id] = len(outlinks)

181761it [02:20, 1297.73it/s]


In [5]:
def page_rank(inlinks_dict, outlinks_dict, outlinks_len, max_iter, d=0.85):
    P = set(inlinks_dict.keys())
    N = len(P)
    PR = {page: 1/N for page in P}

    S = [page for page in P if page not in outlinks_dict]
    L = outlinks_len

    new_PR = {}
    perplexity = 0
    no_change_in_row = 0
    for i in range(max_iter):
        sink_PR = 0
        for page in S:
            sink_PR += PR[page]
        
        for page in P:
            new_PR[page] = (1 - d) / N
            new_PR[page] += d * sink_PR / N
            for q in inlinks_dict[page]:
                if q in PR and L[q] !=0:
                    new_PR[page] += d * PR[q] / L[q]    
        for page in P:
            PR[page] = new_PR[page]
        new_perplexity = 2 ** (-sum(PR[page] * math.log(PR[page],2) for page in P))
        
        print("Iteration: ", i, " Perplexity: ", new_perplexity)
        # Check for convergence
        if int(perplexity) == int(new_perplexity):
            no_change_in_row += 1
        else:
            no_change_in_row = 0
        perplexity = new_perplexity
        if no_change_in_row > 10:
            break

    return PR

def display_top_n(PR, n):
    sorted_PR = sorted(PR.items(), key=lambda x: x[1], reverse=True)[:n]
    print("Top ", n, " pages:")
    for page in sorted_PR:
        print(page[0], ":", page[1])

def save_top_n(PR, n, filename):
    sorted_PR = sorted(PR.items(), key=lambda x: x[1], reverse=True)[:n]
    for page in sorted_PR:
        with open(filename, "a") as f:
            f.write(page[0] + " " + str(page[1]) + " " + str(outlinks_len[page[0]]) + " " + str(inlinks_len[page[0]])+ "\n")

PR = page_rank(inlinks_dict, outlinks_dict, outlinks_len, max_iter=1000)

Iteration:  0  Perplexity:  236.8207213138238
Iteration:  1  Perplexity:  102.57266143539763
Iteration:  2  Perplexity:  63.511036685199265
Iteration:  3  Perplexity:  48.34023021113928
Iteration:  4  Perplexity:  41.500305776960545
Iteration:  5  Perplexity:  37.95716634953101
Iteration:  6  Perplexity:  35.96337441064139
Iteration:  7  Perplexity:  34.77791146482168
Iteration:  8  Perplexity:  34.04329117638761
Iteration:  9  Perplexity:  33.5738650444443
Iteration:  10  Perplexity:  33.26654741008135
Iteration:  11  Perplexity:  33.06144702103438
Iteration:  12  Perplexity:  32.92240583450626
Iteration:  13  Perplexity:  32.826917774826526
Iteration:  14  Perplexity:  32.76061917519963
Iteration:  15  Perplexity:  32.71415407051849
Iteration:  16  Perplexity:  32.68132238907332
Iteration:  17  Perplexity:  32.6579563059339
Iteration:  18  Perplexity:  32.64121954796218
Iteration:  19  Perplexity:  32.62916153980943
Iteration:  20  Perplexity:  32.62042840266198
Iteration:  21  Perpl

In [15]:
def save_top_n(PR, n, filename):
    sorted_PR = sorted(PR.items(), key=lambda x: x[1], reverse=True)[:n]
    for page in sorted_PR:
        with open(filename, "a") as f:
            f.write(page[0] + " " + str(page[1]) + " " + str(outlinks_len[page[0]]) + " " + str(inlinks_len[page[0]])+ "\n")

display_top_n(PR, 500)
save_top_n(PR, 500, "crawl_PageRank.txt")

Top  500  pages:
https://clinicaltrials.gov/policy/reporting-requirements : 0.0009188896909914765
https://oxfordmosaic.web.ox.ac.uk/ : 0.00021349195942021377
https://wikimediafoundation.org/ : 0.00018704181840109756
https://www.usa.gov/ : 0.00012332931218145905
https://www.nih.gov/ : 0.0001203173283145319
https://www.mediawiki.org/wiki/MediaWiki : 0.00011476168076903736
https://www.cornell.edu/ : 0.00010881016250719926
https://developer.wikimedia.org/ : 0.00010715878602261071
https://risr.global/acl_users/credentials_cookie_auth/require_login?came_from=https%3A//risr.global/haiku : 0.00010625351658637616
https://www.newscorporatesubscriptions.com.au/ : 0.0001058861670165313
https://www.bell.ca/Security_and_privacy/Commitment_to_privacy : 0.00010029342529959187
https://github.com/ : 9.880350145679626e-05
https://privacy.cornell.edu/ : 9.734649686131618e-05
https://www.bellmedia.ca/ : 9.128393893939088e-05
https://www.ox.ac.uk/about/facts-and-figures/dates-of-term : 9.05383946598718e-05


In [47]:
# Load the crawl_page_rank into a pandas dataframe
df = pd.read_csv("crawl_PageRank.txt", sep=" ", header=None)
df.columns = ["URL", "PageRank", "Outlinks", "Inlinks"]
df['PageRank'] = df['PageRank'] * 100000
df.head(20)

,URL,PageRank,Outlinks,Inlinks
0,https://clinicaltrials.gov/policy/reporting-re...,91.888969,1,190
1,https://oxfordmosaic.web.ox.ac.uk/,21.349196,44,5805
2,https://wikimediafoundation.org/,18.704182,71,7096
3,https://www.usa.gov/,12.332931,5,3588
4,https://www.nih.gov/,12.031733,81,3506
5,https://www.mediawiki.org/wiki/MediaWiki,11.476168,85,4485
6,https://www.cornell.edu/,10.881016,85,2041
7,https://developer.wikimedia.org/,10.715879,8,4233
8,https://risr.global/acl_users/credentials_cook...,10.625352,17,7347
9,https://www.newscorporatesubscriptions.com.au/,10.588617,0,521


In [53]:
index = 0
highest_rank = df['URL'][index]
print("Highest PageRank URL: ", highest_rank)
print("Highest PageRank: ", df['PageRank'][index])

print("Getting the inlinks of the highest PageRank URL")
print("Got ", len(inlinks_dict[highest_rank]), " inlinks")

# Get the average page rank of all the inlinks
avg_PR = 0
for inlink in inlinks_dict[highest_rank]:
    print(inlink, PR[inlink]*100000)
    avg_PR += PR[inlink] * 100000
avg_PR = avg_PR / len(inlinks_dict[highest_rank])
print("Average PageRank of inlinks: ", avg_PR)

Highest PageRank URL:  https://clinicaltrials.gov/policy/reporting-requirements
Highest PageRank:  91.88896909914
Getting the inlinks of the highest PageRank URL
Got  190  inlinks
https://clinicaltrials.gov:443/search?cond=%22Dementia%22&aggFilters=status%3Anot+rec+ava 0.08956820774416721
https://clinicaltrials.gov/study/NCT04192500 0.0826078987021847
https://clinicaltrials.gov/search?cond=%22Headache+Disorders%22&aggFilters=status:not%20rec 0.08373397399186093
https://clinicaltrials.gov:443/search?cond=Sjogren%27s+Syndrome&city= 0.08582920137713476
https://clinicaltrials.gov/study/NCT01142362 0.0826078987021847
https://clinicaltrials.gov/study/NCT03293498 0.0826078987021847
https://clinicaltrials.gov/study/NCT05426174 0.0826078987021847
https://clinicaltrials.gov/study/NCT05827978 0.0826078987021847
https://clinicaltrials.gov:443/search?cond=Rett+Syndrome&aggFilters=status%3Arec+act 0.08617622440054049
https://clinicaltrials.gov/study/NCT05155319 0.0826078987021847
https://clinicaltri